In [5]:
from pyspark.sql import SparkSession

In [6]:
spark=SparkSession.builder.appName("Analyze London crime data").getOrCreate()

In [7]:
data=spark.read\
        .format("csv")\
        .option("header", "true")\
        .load("..\\datasets\london_crime_by_lsoa.csv")

In [8]:
data.printSchema()

root
 |-- lsoa_code: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- major_category: string (nullable = true)
 |-- minor_category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)



In [9]:
data.count()

13490604

In [10]:
data.limit(5).show()

+---------+----------+--------------------+--------------------+-----+----+-----+
|lsoa_code|   borough|      major_category|      minor_category|value|year|month|
+---------+----------+--------------------+--------------------+-----+----+-----+
|E01001116|   Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|E01001646| Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|E01000677|   Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|E01003774| Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|E01004563|Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
+---------+----------+--------------------+--------------------+-----+----+-----+



In [11]:
data.dropna()

DataFrame[lsoa_code: string, borough: string, major_category: string, minor_category: string, value: string, year: string, month: string]

In [12]:
data=data.drop("lsoa_code")

In [13]:
data.show()

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|             Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|           Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|             Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|           Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|          Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
|              Ealing|  Theft and Handling|         Other Theft|    0|2012|    5|
|              Ealing|Violence Against ...|    Offensive Weapon|    0|2010|    7|
|            Hounslow|             Robbery|   Personal Property|    0|2013|    4|
|              Newham|     Criminal Damage|Criminal Damage T...|    0|2013|    9|
|              S

In [14]:
total_boroughs=data.select('borough').distinct()
total_boroughs.show()

+--------------------+
|             borough|
+--------------------+
|             Croydon|
|          Wandsworth|
|              Bexley|
|             Lambeth|
|Barking and Dagenham|
|              Camden|
|           Greenwich|
|              Newham|
|       Tower Hamlets|
|            Hounslow|
|              Barnet|
|              Harrow|
|Kensington and Ch...|
|           Islington|
|               Brent|
|            Haringey|
|             Bromley|
|              Merton|
|         Westminster|
|             Hackney|
+--------------------+
only showing top 20 rows



In [15]:
total_boroughs.count()

33

In [16]:
hackney_data=data.filter(data['borough'] == "Hackney")
hackney_data.show()

+-------+--------------------+--------------------+-----+----+-----+
|borough|      major_category|      minor_category|value|year|month|
+-------+--------------------+--------------------+-----+----+-----+
|Hackney|     Criminal Damage|Criminal Damage T...|    0|2011|    6|
|Hackney|Violence Against ...|          Harassment|    1|2013|    2|
|Hackney|     Criminal Damage|Other Criminal Da...|    0|2011|    7|
|Hackney|Violence Against ...|        Wounding/GBH|    0|2013|   12|
|Hackney|  Theft and Handling|  Other Theft Person|    0|2016|    8|
|Hackney|            Burglary|Burglary in a Dwe...|    2|2008|    5|
|Hackney|             Robbery|   Business Property|    0|2016|    7|
|Hackney|  Theft and Handling|Theft/Taking of P...|    0|2009|   12|
|Hackney|               Drugs|    Drug Trafficking|    0|2014|    4|
|Hackney|  Theft and Handling|Handling Stolen G...|    0|2014|    6|
|Hackney|            Burglary|Burglary in Other...|    0|2008|   12|
|Hackney|Violence Against ...| Ass

In [17]:
data_2015_2016=data.filter(data['year'].isin(["2015","2016"]))
data_2015_2016.sample(fraction=0.1).show()

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|           Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|              Sutton|  Theft and Handling|Theft/Taking of P...|    1|2016|    8|
|      Waltham Forest|  Theft and Handling|Motor Vehicle Int...|    0|2016|    3|
|           Islington|  Theft and Handling|Theft/Taking Of M...|    0|2016|    6|
|Kingston upon Thames|  Theft and Handling|Motor Vehicle Int...|    0|2016|    5|
|            Hounslow|Violence Against ...|    Offensive Weapon|    1|2015|   10|
|      Waltham Forest|            Burglary|Burglary in Other...|    0|2015|    6|
|Kingston upon Thames|Other Notifiable ...|      Going Equipped|    0|2015|   12|
|               Brent|Other Notifiable ...|    Other Notifiable|    0|2016|    4|
|          Hilli

In [18]:
boro_crime_cnt=data.groupBy('borough').count()
boro_crime_cnt.show(5)

+--------------------+------+
|             borough| count|
+--------------------+------+
|             Croydon|602100|
|          Wandsworth|498636|
|              Bexley|385668|
|             Lambeth|519048|
|Barking and Dagenham|311040|
+--------------------+------+
only showing top 5 rows



In [19]:
boro_convic_sum=data.groupBy('borough').agg({"value":"sum"}).withColumnRenamed("sum(value)", "convictions")
boro_convic_sum.show(5)

+--------------------+-----------+
|             borough|convictions|
+--------------------+-----------+
|             Croydon|   260294.0|
|          Wandsworth|   204741.0|
|              Bexley|   114136.0|
|             Lambeth|   292178.0|
|Barking and Dagenham|   149447.0|
+--------------------+-----------+
only showing top 5 rows



In [20]:
total_boro_convics=boro_convic_sum.agg({"convictions":"sum"})
total_boro_convics.show()

+----------------+
|sum(convictions)|
+----------------+
|       6447758.0|
+----------------+



In [21]:
total_convics=total_boro_convics.collect()[0][0]

In [22]:
import pyspark.sql.functions as func

In [23]:
boro_pc_contr=boro_convic_sum.withColumn("% contribution", func.round(boro_convic_sum.convictions/total_convics*100,2))
boro_pc_contr.printSchema()

root
 |-- borough: string (nullable = true)
 |-- convictions: double (nullable = true)
 |-- % contribution: double (nullable = true)



In [24]:
boro_pc_contr.orderBy(boro_pc_contr[2].desc()).show(10)

+-------------+-----------+--------------+
|      borough|convictions|% contribution|
+-------------+-----------+--------------+
|  Westminster|   455028.0|          7.06|
|      Lambeth|   292178.0|          4.53|
|    Southwark|   278809.0|          4.32|
|       Camden|   275147.0|          4.27|
|       Newham|   262024.0|          4.06|
|      Croydon|   260294.0|          4.04|
|       Ealing|   251562.0|           3.9|
|    Islington|   230286.0|          3.57|
|Tower Hamlets|   228613.0|          3.55|
|        Brent|   227551.0|          3.53|
+-------------+-----------+--------------+
only showing top 10 rows



In [25]:
convic_mthly=data.filter(data['year'] == 2014).groupBy('month').agg({"value":"sum"}).withColumnRenamed("sum(value)","convictions")

In [32]:
convic_mthly.orderBy(convic_mthly.convictions.desc()).show()

+-----+-----------+
|month|convictions|
+-----+-----------+
|   10|    60537.0|
|   11|    59704.0|
|    7|    58564.0|
|    3|    57669.0|
|   12|    57565.0|
|    6|    57039.0|
|    9|    56933.0|
|    5|    56327.0|
|    8|    55641.0|
|    1|    55515.0|
|    4|    53467.0|
|    2|    51222.0|
+-----+-----------+



In [30]:
crime_cat=data.groupBy('major_category').agg({"value":"sum"}).withColumnRenamed("sum(value)", "convictions")

In [31]:
crime_cat.orderBy(crime_cat.convictions.desc()).show()

+--------------------+-----------+
|      major_category|convictions|
+--------------------+-----------+
|  Theft and Handling|  2661861.0|
|Violence Against ...|  1558081.0|
|            Burglary|   754293.0|
|     Criminal Damage|   630938.0|
|               Drugs|   470765.0|
|             Robbery|   258873.0|
|Other Notifiable ...|   106349.0|
|    Fraud or Forgery|     5325.0|
|     Sexual Offences|     1273.0|
+--------------------+-----------+



In [33]:
year_df=data.select('year')

In [34]:
year_df.agg({"year":"min"}).show()

+---------+
|min(year)|
+---------+
|     2008|
+---------+



In [35]:
year_df.agg({"year":"max"}).show()

+---------+
|max(year)|
+---------+
|     2016|
+---------+



In [36]:
year_df.describe().show()

+-------+-----------------+
|summary|             year|
+-------+-----------------+
|  count|         13490604|
|   mean|           2012.0|
| stddev|2.581988993167432|
|    min|             2008|
|    max|             2016|
+-------+-----------------+



In [38]:
data.crosstab('borough','major_category').select('borough_major_category', 'Burglary', 'Drugs').show()

+----------------------+--------+-----+
|borough_major_category|Burglary|Drugs|
+----------------------+--------+-----+
|              Havering|   32400|32616|
|                Merton|   26784|29160|
|              Haringey|   31320|35424|
|         Tower Hamlets|   31104|37368|
|               Bromley|   42552|42336|
|               Enfield|   39528|44064|
|  Kingston upon Thames|   21168|22140|
|           Westminster|   27648|32616|
|  Richmond upon Thames|   24840|23004|
|              Lewisham|   36504|43740|
|                 Brent|   37368|46980|
|  Barking and Dagenham|   23760|26244|
|             Redbridge|   34776|36504|
|             Islington|   26568|34128|
|               Hackney|   31104|38772|
|                Newham|   35424|41580|
|        City of London|     540|  756|
|                Sutton|   26136|26784|
|             Southwark|   35856|45144|
|                Harrow|   29592|31212|
+----------------------+--------+-----+
only showing top 20 rows



In [52]:
data.filter(data.year == 2015).groupBy('borough').agg({'value':'sum'}).show()

+--------------------+----------+
|             borough|sum(value)|
+--------------------+----------+
|             Croydon|   28089.0|
|          Wandsworth|   22401.0|
|              Bexley|   11990.0|
|             Lambeth|   33164.0|
|Barking and Dagenham|   16346.0|
|              Camden|   29477.0|
|           Greenwich|   20929.0|
|              Newham|   29217.0|
|       Tower Hamlets|   27493.0|
|            Hounslow|   21286.0|
|              Barnet|   24036.0|
|              Harrow|   12601.0|
|Kensington and Ch...|   19070.0|
|           Islington|   27032.0|
|               Brent|   24883.0|
|            Haringey|   24626.0|
|             Bromley|   20637.0|
|              Merton|   12273.0|
|         Westminster|   47395.0|
|           Southwark|   30260.0|
+--------------------+----------+
only showing top 20 rows



In [40]:
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [54]:
""""def describe_year(year):
    yearly_details = data.filter(data.year == year).groupBy('borough').agg({'value':'sum'}).withColumnRenamed("sum(value)", "convictions")
    boro_list = [x[0] for x in yearly_details.toLocalIterator()]
    convic_list = [x[1] for x in yearly_details.toLocalIterator()]
    plt.figure(figsize=(33,10))
    plt.bar(boro_list, convic_list)
    plt.show()
    plt.xlabel('Boroughs',fontsize=30)
    plt.ylabel('Convictions', fontsize=30)
    plt.xticks(rotation=90,fontsize=30)
    plt.yticks(fontsize=30)
    plt.autoscale()
    plt.show()""
    
    

In [58]:
""""describe_year(2015)"""

'"describe_year(2015)'